In [17]:
!pip install selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:13 http://ppa.launchpad

In [20]:
import requests
from lxml import html, etree
from selenium.webdriver import Chrome, ChromeOptions
import random
import time
from datetime import datetime, timedelta
import csv

In [21]:
def data_writer(li):
    with open(r'output.csv', mode="a",
              newline="", encoding="utf-8-sig") as f_out:
        my_writer = csv.writer(f_out, delimiter=',')
        my_writer.writerow(li)
data_writer(['URL', "Title", "Location",
            "No. of Reviews", "Rating", "Price Range", "Cuisine"])

In [22]:
BASE = 'https://www.tripadvisor.com'

##Chrome Driver Options##
width = range(500, 700, 10)
height = range(800, 1000, 20)
chrome_options = ChromeOptions()
chrome_options.add_argument(
    f"window-size={random.choice(width)},{random.choice(height)}")
chrome_options.add_argument('--no-default-browser-check')
chrome_options.add_argument('--disable-gpu')
chrome_options.add_argument('--disable-extensions')
chrome_options.add_argument('--disable-default-apps')
chrome_options.add_argument('--disable-notifications')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('--log-level=3')
chrome_options.add_argument('--headless')
wd = Chrome('chromedriver',options=chrome_options)
wd.get('https://www.tripadvisor.com/Restaurants-g187849-Milan_Lombardy.html')
try:
    wd.find_element_by_xpath('//button[@id="_evidon-accept-button"]').click()
except:
    pass
time.sleep(2)
wd.find_element_by_xpath('//span[contains(text(), "Takeout")]').click()
time.sleep(3)
tree = html.fromstring(wd.page_source)

In [23]:
for item in tree.xpath('//div[@data-component-props="page-manifest"]/div//span/text()'):
    try:
        num_results = int(item)
        break
    except:
        pass
a_li = [0]
a = 0
for i in range(int(num_results/30)):
    a += 30
    a_li.append(a)

In [24]:
headers = {
    'authority': 'www.tripadvisor.com',
    'sec-ch-ua': '"Chromium";v="92", " Not A;Brand";v="99", "Google Chrome";v="92"',
    'x-requested-with': 'XMLHttpRequest',
    'sec-ch-ua-mobile': '?0',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 Safari/537.36',
    'accept': '*/*',
    'sec-fetch-site': 'same-origin',
    'sec-fetch-mode': 'cors',
    'sec-fetch-dest': 'empty',
    'referer': 'https://www.tripadvisor.com/Restaurants-g187849-Milan_Lombardy.html',
    'accept-language': 'en-US,en;q=0.9,ar;q=0.8',
    "cookie":''
}

page_counter = 1
for a in a_li:
    params = (
    ('Action', 'PAGE'),
    ('ajax', '1'),
    ('availSearchEnabled', 'true'),
    ('sortOrder', 'relevance'),
    ('geo', '187849'),
    ('itags', '10591'),
    ('zfp', '10601'),
    ('eaterydate', datetime.today().strftime('%Y_%m_%d')),
    ('date', (datetime.today()+timedelta(days=1)).strftime('%Y-%m-%d')),
    ('time', '20:00:00'),
    ('people', '2'),
    ('o', f'a{a}'),
    )
    r = requests.get(f'https://www.tripadvisor.com/RestaurantSearch', params=params, headers=headers)
    tree = html.fromstring(r.content)
    list_of_restaurants = tree.xpath('//div[contains(@data-test,"list_item")]')
    for restaurant in list_of_restaurants:
        restaurant_url = f"{BASE}{restaurant.xpath('.//a')[0].attrib['href']}"
        wd.get(restaurant_url)
        tree = html.fromstring(wd.page_source)
        restaurant = tree.xpath('//div[contains(@id,"taplc_top_info")]')[0]
        try:
            restaurant_name = restaurant.xpath('.//h1')[0].text
            for item in restaurant.xpath('.//a'):
                try:
                    item_href = item.attrib['href']
                    if item_href == '#MAPVIEW':
                        restaurant_location = item.text
                    if item_href == '#REVIEWS':
                        restaurant_reviews_num = item.xpath('.//span')[0].text
                        restaurant_reviews_rating = item.xpath(
                            './/svg')[0].attrib['title'].split(' ')[0]
                except:
                    pass
            try:
                price = tree.xpath(
                    '//div[contains(text(),"PRICE RANGE")]/following-sibling::div')[0].text
            except:
                price = ''
            try:
                cuisine = tree.xpath(
                '//div[contains(text(),"CUISINES")]/following-sibling::div')[0].text
            except:
                cuisine = ''
            data_writer([restaurant_url, restaurant_name, restaurant_location,
                        restaurant_reviews_num, restaurant_reviews_rating, price, cuisine])
            print(f'{restaurant_name} Scrapped.')
        except:
            print(restaurant_url)
    print(f'Page No. {page_counter} scrapped')
    print()
    page_counter += 1

Mabuhay Restaurant Scrapped.
Star Zagros Kebabbar Scrapped.
Shabby Grill Restaurant Scrapped.
Il Panino del Laghetto Scrapped.
Panzarotti Scrapped.
Pizzium - Via Vigevano Scrapped.
Linfa Milano - Eat Different Scrapped.
La Tartina Scrapped.
Piadineria Artigianale Pascoli Scrapped.
Yokohama Flavour Journey Cuisine Scrapped.
Kate Away Milano Scrapped.
Tram Scrapped.
Viaggi Nel Gusto Scrapped.
l’Anlot e Oltre Scrapped.
Ristorante Xiongdi Scrapped.
Nàpiz' Milano Scrapped.
La TASTeria Gourmet Sicily Milano Brera Scrapped.
Pomet Milano Scrapped.
Piadineria Doppio Zero Scrapped.
Pasta d'Autore Scrapped.
Cibario Scrapped.
Olo-olo Ravioleria Scrapped.
Soprattutto Scrapped.
LUX Gourmet Social Club Scrapped.
Pizza AM Scrapped.
G sushi - Oriental Pleasure Scrapped.
Pizzeria Cinder Mount Scrapped.
Taac Trattoria Scrapped.
Pizzium - Stazione Centrale Milano Scrapped.
Sapori Solari - Salumeria Bistrot Scrapped.
Page No. 1 scrapped

Ristorante Pizzeria O Surdato 'Nnamurato Scrapped.
Testone Viale Blig